In [ ]:
#load zip file
import zipfile
import os
import glob
out='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/raw/'
zips_folder='/home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/'
zips = glob.glob(zips_folder+'*')
for zip_file in zips:
    try:
        with zipfile.ZipFile(zip_file+'/json_000001.zip', 'r') as zip_ref:
            zip_ref.extractall(out)
    except:
        print('erro no arquivo: '+zip_file)
        continue

erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/3704569-Renata Carvalho-FINAL cd
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/6711289-IVANEI-DE-CARVALHO-DOS-SANTOS
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/5003499-kubota_tyk_dr_ilha
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/5126255-Tese doutorado_CAroline Robinson 2018-1
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/1537352-Silva, Edson Batista da
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/6080441-DISSERTAÇÃO-MAURICIO-completa
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/6426050-Thesis final-Adriane Hauer_05
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/4573176-KATIA BILHAR SCAPINI
erro no arquivo: /home/kiki/dados_servidor/Servidor/Teses/samples_deepsearch/5093617-LOPES, Cleber
er

In [ ]:
# !rm /home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/bboxes/*
# !rm -r /home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/images/*

In [ ]:
# !zip -r deepsearch_dataset.zip /home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset

In [ ]:
import json
import pathlib
import pdfplumber


json_dir = "/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/raw/"
pdf_dir = '/home/kiki/HD/Teses/samples/'

out_img_dir='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/images/'
out_bboxes_dir='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/bboxes/'


def get_pdf_pages(pdf_file: str):
    pdf = pdfplumber.open(pdf_file)
    pages = []
    for page in pdf.pages:
        pages.append(page)
    return pages

def get_bbox(annotations,page_number: int = 1):
    bbox = []
    page_dim = annotations['page-dimensions'][page_number - 1]
    for annot in annotations['main-text']:
        if "$ref" in annot:
            ref = annot['$ref'].split('/')[-1]
            annot_type = annot['type'] + 's'
            if annot_type in annotations:
                annot = annotations[annot_type][int(ref)]

        if annot['prov'][0]['page'] == page_number:
            x0, y0, x1, y1 = annot['prov'][0]['bbox']
            bbox.append({"bbox": (
                x0, page_dim['height'] - y1, x1, page_dim['height'] - y0
            ),"bbox_coords":(x0,y0,x1,y1), "type": annot['type']})
                
    return bbox

def extract():
    paths = pathlib.Path(json_dir).glob('*.json')
    c=0
    for path in paths:
        c+=1
        if c%100==0 and c!=0:
            print(c)
        try:
            #check if dir already exists
            if pathlib.Path(out_img_dir+path.stem).exists():
                continue
            annotations = json.loads(path.read_text())
            pdf_file = pdf_dir + path.stem + '.pdf'
            pages = get_pdf_pages(pdf_file)

            #get bounding boxes
            bboxes=[]
            bboxes_coords=[]
            bboxes_type=[]
            for i in range(len(pages)):
                
                result = get_bbox(annotations,page_number=i+1)
                bbox = [r['bbox'] for r in result]
                bbox_type = [r['type'] for r in result]
                bbox_coords = [r['bbox_coords'] for r in result]
                bboxes.append(bbox)
                bboxes_type.append(bbox_type)
                bboxes_coords.append(bbox_coords)

            #get and save images
            if not pathlib.Path(out_img_dir+path.stem).exists():
                pathlib.Path(out_img_dir+path.stem).mkdir()

            for n in range(len(pages)):
                print('file:',c,'page: ',n)
                p=pages[n]
                im = p.to_image(resolution=150)
                im.save(out_img_dir+path.stem+'/'+path.stem+'_'+str(n+1)+'.jpg')
            
            #save bboxes and bboxes types in json with pages
            pages_bboxes_json={}
            for n in range(len(pages)):
                pages_bboxes_json[str(n+1)]=[{'bbox':bboxes[n][i],'bbox_coords':bboxes_coords[n][i],'type':bboxes_type[n][i]} for i in range(len(bboxes[n]))]
            with open(out_bboxes_dir+path.stem+'.json', 'w') as fp:
                json.dump(pages_bboxes_json, fp)
        except KeyboardInterrupt:
            break
        except Exception as e:
            print('error in '+str(path))
            print(e)
            
            continue

In [ ]:
extract()

file: 91 page:  0
file: 91 page:  1
file: 91 page:  2
file: 91 page:  3
file: 91 page:  4
file: 91 page:  5
file: 91 page:  6
file: 91 page:  7
file: 91 page:  8
file: 91 page:  9
file: 91 page:  10
file: 91 page:  11
file: 91 page:  12
file: 91 page:  13
file: 91 page:  14
file: 91 page:  15
file: 91 page:  16
file: 91 page:  17
file: 91 page:  18
file: 91 page:  19
file: 91 page:  20
file: 91 page:  21
file: 91 page:  22
file: 91 page:  23
file: 91 page:  24
file: 91 page:  25
file: 91 page:  26
file: 91 page:  27
file: 91 page:  28
file: 91 page:  29
file: 91 page:  30
file: 91 page:  31
file: 91 page:  32
file: 91 page:  33
file: 91 page:  34
file: 91 page:  35
file: 91 page:  36
file: 91 page:  37
file: 91 page:  38
file: 91 page:  39
file: 91 page:  40
file: 91 page:  41
file: 91 page:  42
file: 91 page:  43
file: 91 page:  44
file: 91 page:  45
file: 91 page:  46
file: 91 page:  47
file: 91 page:  48
file: 91 page:  49
file: 91 page:  50
file: 91 page:  51
file: 91 page:  52
fil

[WARNING] Metadata key "CreationDate" could not be parsed due to exception: maximum recursion depth exceeded while calling a Python object


file: 499 page:  0
file: 499 page:  1
file: 499 page:  2
file: 499 page:  3
file: 499 page:  4
file: 499 page:  5
file: 499 page:  6
file: 499 page:  7
file: 499 page:  8
file: 499 page:  9
file: 499 page:  10
file: 499 page:  11
file: 499 page:  12
file: 499 page:  13
file: 499 page:  14
file: 499 page:  15
file: 499 page:  16
file: 499 page:  17
file: 499 page:  18
file: 499 page:  19
file: 499 page:  20
file: 499 page:  21
file: 499 page:  22
file: 499 page:  23
file: 499 page:  24
file: 499 page:  25
file: 499 page:  26
file: 499 page:  27
file: 499 page:  28
file: 499 page:  29
file: 499 page:  30
file: 499 page:  31
file: 499 page:  32
file: 499 page:  33
file: 499 page:  34
file: 499 page:  35
file: 499 page:  36
file: 499 page:  37
file: 499 page:  38
file: 499 page:  39
file: 499 page:  40
file: 499 page:  41
file: 499 page:  42
file: 499 page:  43
file: 499 page:  44
file: 499 page:  45
file: 499 page:  46
file: 499 page:  47
file: 499 page:  48
file: 499 page:  49
file: 499 